# Explore

- `random_state`=123

Plan
- Split data
- Get Univariate Insights
- Get Bivariate Insights to target (quality)
    - scatterplots with quality on the y
    - correlations + heatmap
    - barplot, swarmplot, and/or boxplot with quality on y and color on x